# Function 1: Check Images in folder for QR Codes and EAN13 or ITF and EAN13

In [ ]:
from PIL import Image
from pyzbar.pyzbar import decode
import os

def QR_Checker(directory):
    qr_count = 0
    qr_codes = []
    ean_count = 0
    ean_codes = []
    
    for i in os.listdir(directory):
        pic_dir = (directory + i)
        img = Image.open(pic_dir)
        #exploring greyscale vs normal images
        #img = img.convert(mode = "1", dither = Image.NONE)
        code = decode(img)
            
        for barcode in code:
            if barcode.type == 'QRCODE':
                qr_code = barcode.data.decode('utf-8')
                qr_codes.append(qr_code)
                qr_count += 1

            elif barcode.type == 'EAN13':
                ean_code = barcode.data.decode('utf-8')
                ean_codes.append(ean_code)
                ean_count += 1
    
    return qr_count, qr_codes, ean_count, ean_codes

In [ ]:
def ITF_Checker(directory):
    itf_count = 0
    itf_codes = []
    ean_count = 0
    ean_codes = []
    
    for i in os.listdir(directory):
        pic_dir = (directory + i)
        img = Image.open(pic_dir)
        #explore greyscale vs normal images
        #img = img.convert(mode = "1", dither = Image.NONE)
        code = decode(img)
            
        for barcode in code:
            if barcode.type == 'I25':
                itf_code = barcode.data.decode('utf-8')
                itf_codes.append(itf_code)
                itf_count += 1

            elif barcode.type == 'EAN13':
                ean_code = barcode.data.decode('utf-8')
                ean_codes.append(ean_code)
                ean_count += 1
    
    return itf_count, itf_codes, ean_count, ean_codes

# Function 2: Comparing decoded QR Codes (should be different for all 3) or ITF Codes (should be the same for both)

In [ ]:
import os
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def QR_Test(lst):
    test = {"check 1": [],
           "check 2": [],
           "check 3": [],
           "check 4": []}
    
    verdict = 'Passed QR Check'
    
    if len(lst) != 3:
        verdict = 'Error'
        return test, verdict
    
    qr_1 = (lst[0].split('/'))[-1]
    qr_2 = (lst[1].split('/'))[-1]
    qr_3 = (lst[2].split('/'))[-1]
    
    
    #Check level 1 (Correct Nike Website)
    front = 'HTTPS://QR.NIKE.COM/'
    count = 0
    while count < len(lst):
        if not front in lst[count]:
            status = 'Fail: QR ' + str(count+1) + ' is not the proper Nike Website' 
            test["check 1"].append(status)
            count += 1
        else:
            count += 1
     
    if (len(test["check 1"]) == 0):
        test["check 1"].append('Pass: QR Codes lead to proper Nike Website')
    
    
    #Check level 2 (Correct number of values)
    if len(qr_1) == 13:
        if len(qr_2) == 13:
            if len(qr_3) == 13:
                test["check 2"].append('Pass: Correct number of values in QR code')
            else:
                test["check 2"].append('Fail: QR 3 has incorrect number of values')
        else:
            test["check 2"].append('Fail: QR 2 has incorrect number of values')
    else:
        test["check 2"].append('Fail: QR 1 has incorrect number of values')
    
    
    #Check level 3
    if (qr_1 == qr_2):
        test["check 3"].append('Fail: QR Code 1 and QR Code 2 are the same')
    elif (qr_1 == qr_3):
        test["check 3"].append('Fail: QR Code 1 and QR Code 3 are the same')
    elif (qr_2 == qr_3):
        test["check 3"].append('Fail: QR Code 2 and QR Code 3 are the same')
    else:
        test["check 3"].append('Pass: Unique QR Code')
        
        
    #Check level 4
    if ((round((similarity(qr_1,qr_2)*100),2)) > 50):
        test["check 4"].append('Fail: >50% similarity in QR 1 and QR 2')
        
    elif ((round((similarity(qr_1,qr_3)*100),2)) > 50):
        test["check 4"].append('Fail: >50% similarity in QR 1 and QR 3')
    
    elif ((round((similarity(qr_2,qr_3)*100),2)) > 50):
        test["check 4"].append('Fail: >50% similarity in QR 2 and QR 3')
        
    else:
        test["check 4"].append('Pass: <30% similarity of QR Code')
    
    for i in test.values():
        if 'Fail' in i[0]:
            verdict = 'Failed QR Check'
    
    for i in test.values():
        if 'Fail' in i[0]:
            verdict = 'Failed QR Check'
            
    return test, verdict

In [ ]:
def ITF_Test(lst):
    test = {"check 1": [],
           "check 2": []}
    
    verdict = 'Passed ITF Check'
    
    #Check 1: Length of decoded ITF should be 10
    if len(lst[0]) == 10:
        if len(lst[1]) == 10:
            test["check 1"].append("Pass: ITF Codes are the correct number of characters")
        else:
            test["check 1"].append("Fail: ITF 2 is not the correct number of characters")
    else:
        test["check 1"].append("Fail: ITF 1 is not the correct number of characters")
        
    #Check 2: Decoded ITF should be the same
    if lst[0] == lst[1]:
        test["check 2"].append("Pass: ITF Codes are the same")
    else:
        test["check 2"].append("Fail: ITF Codes are not the same")
        
    for i in test.values():
        if 'Fail' in i[0]:
            verdict = 'Failed QR Check'    
        
    return test, verdict

# Function 3: OCR (Look for decoded QR code on the size tag)

In [ ]:
import pytesseract as tess
from PIL import Image
import os
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

#This function splits up words picked out from OCR into a list
def Image_Cleaner(path):
    img = Image.open(path)
    greyscale_img = img.convert(mode = "1", dither = Image.NONE)
    text = tess.image_to_string(img)
    text = text.replace(' ', '\n')
    text = text.split('\n')
    
    return text

#input image path and dictionary (keys: SKU, decoded QR Codes, UPC) of details to find in the OCR-ed images
def Details_Finder(img, dets):
    text = Image_Cleaner(img)
    
    SKU = None
    UPC = None
    QR = None
    
    for i in text:
        for j in dets["QR"]:
            if i == dets["SKU"]:
                SKU = i
            elif i == dets["UPC"]:
                UPC = i
            elif i == j:
                QR = i
    
    return QR, UPC, SKU

# To run the 4 programs:
## 1) Input file path (10 Airtable images)

In [ ]:
path = r"C:\Users\hakim\Documents\Novelship Work\Images\For Testing\Test (QR)\\"

## 2) Run following code to iterate through folder and check number of QR Codes, EAN detected

### 2.1 Actual output

In [ ]:
print(QR_Checker(path))

### 2.2 Cleaned output

In [ ]:
qr_count, qr_codes, ean_count, ean_codes = QR_Checker(path)
print('Number of QR Codes detected: ' + str(qr_count))
print('Number of EAN detected: ' + str(ean_count))

## 3) Check QR Codes and EAN detected (3 and 1 respectively)

### 3.1 Actual view

In [ ]:
print(qr_codes)
print(ean_codes)

### 3.2 Cleaned view

In [ ]:
for i in qr_codes:
    print('QR Code Detected: ' + i)
for j in ean_codes:
    print('EAN Detected: ' + j)

## 4) Run following code to compare decoded QR Codes. 4 Levels of checks

### 4.1 Actual output

In [ ]:
test, verdict = QR_Test(qr_codes)
print(test)
print(verdict)

### 4.2 Cleaned output

In [ ]:
test, verdict = QR_Test(qr_codes)

for i in test.values():
    print(i[0])
print(verdict)

## 5) Input images to be OCR-ed and details to be checked for (Prior to this crop and rotate images according to grid)

In [ ]:
SKU = 'AQ3692-001'
UPC = '00193654541781'
QRL = '000K0JWVF6RQ8'
QRR = '019FSCWS2E83T'

dets = {"SKU": SKU, "UPC": UPC, "QR": [QRL, QRR]}

path = r"C:\Users\hakim\Documents\Novelship Work\Images\For Testing\Test (OCR)\1.jpg"

In [ ]:
QR, UPC, SKU = Details_Finder(path, dets)
print('QR numbers detected: ' + QR)
#print('UPC numbers detected: ' + UPC)
#print('SKU numbers detected: ' + SKU)

In [ ]:
img = Image.open(path)
greyscale_img = img.convert(mode = "1", dither = Image.NONE)
text = tess.image_to_string(img)

In [ ]:
print(text)